# Orient Express &ndash; Planning algorithm

In [1]:
### IMPORTS
import datetime

import networkx as nx
import pandas as pd
from hdfs3 import HDFileSystem

from orientexpress.planner import RoutePlanner
from orientexpress.utils import parse_timestamp, print_timestamp, RouteNode
from orientexpress.delay import Delay

%load_ext autoreload
%autoreload 2

In [2]:
### HDFS COMMUNICATION
hdfs = HDFileSystem(host='hdfs://iccluster044.iccluster.epfl.ch', port=8020, user='ebouille')

transfers = pd.DataFrame()
for path in hdfs.ls('/user/tvaucher/transfers'):
    if not 'SUCCESS' in path:
        with hdfs.open(path) as f:
            transfers = transfers.append(pd.read_parquet(f))
            
edges = pd.DataFrame()
for path in hdfs.ls('/user/tvaucher/edges'):
    if not 'SUCCESS' in path:
        with hdfs.open(path) as f:
            edges = edges.append(pd.read_parquet(f))

stops = pd.DataFrame()
for path in hdfs.ls('/user/tvaucher/stops'):
    if not 'SUCCESS' in path:
        with hdfs.open(path) as f:
            stops = stops.append(pd.read_parquet(f))

len(stops), len(edges), len(transfers)

(1880, 257883, 12534)

## 1. Building the public transport network graph

We want to represent the public transport network as a directed weighted multi-edge graph. Each node represents a stop, with a stop ID. Each edge represents a trip or walking transfer between any two stops (less than 500m from each other).

We construct the graph below, starting by adding nodes, and then both types of edges.

Each **node**, representing a stop, possesses the following attributes:

- `stop_id`: stop identifier
- `stop_name`: Full name of the stop (not unique because different platforms)
- `stop_lat`, `stop_lon`: GPS coordinates of the node location

Each **edge**, representing a trip or walking transfer, possesses the following attributes:

Transfers:
- source node, target node identifiers: `stop_id`
- `distance`: distance between stops, in meters
- `t_time`: transfer time

Trips:
- source node, target node identifiers (consider stop A -> stop B): `stop_id`
- `prev_dep_time`: departure time from stop prior to departure stop id (departure time to arrive at stop A)
- `arr_time`: arrival time at stop B
- `duration`: duration of trip, in seconds
- `dep_time`: departure time from stop A
- `trip_id`: trip identifier
- `trip_headsign`: destination name in which the transport is headed
- `route_desc`: type of transport (Zug, Tram, Bus, etc.)
- `route_short_name`: short name of the route
- `stop_sequence`: number of stop in order of direction the transport is headed

#### 1.1 Nodes

In [3]:
stops.sample(5)

,stop_id,stop_name,stop_lat,stop_lon
1645,8591279,"Zürich, Morgental",47.343948,8.530141
645,8575941,"Lindau, Eschikon",47.448078,8.682289
230,8503102:0:3,Erlenbach ZH,47.305848,8.591505
1717,8591358,"Zürich, Segantinistrasse",47.407446,8.489969
255,8503128:0:4,Dübendorf,47.400137,8.623405


In [4]:
G = nx.MultiDiGraph()
G.add_nodes_from(stops.stop_id)
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 1880
Number of edges: 0
Average in degree:   0.0000
Average out degree:   0.0000


#### 1.2.1 Trips

In [5]:
edges.apply(lambda row: G.add_edge(row['stop_id'], row['prev_stop_id'],
                                   prev_dep_time=parse_timestamp(row['prev_departure_time']),
                                   arr_time=parse_timestamp(row['arrival_time']),
                                   duration=row["duration"],
                                   dep_time=parse_timestamp(row['departure_time']),
                                   trip_id=row['trip_id'],
                                   trip_headsign=row['trip_headsign'],
                                   route_desc=row['route_desc'],
                                   route_short_name=row['route_short_name'],
                                   stop_sequence=row['stop_sequence'],
                                  ), axis=1);

In [6]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 1880
Number of edges: 257883
Average in degree: 137.1718
Average out degree: 137.1718


#### 1.2.2 Transfers

In [7]:
transfers.sample(5)

,from_stop_id,to_stop_id,distance,t_time
70,8591088,8591267,451.446747,661.736096
49,8591050,8591113,471.465973,685.759167
63,8591326,8591292,229.558289,395.469946
50,8503203:0:1,8502209:0:1,333.722839,520.467407
25,8587860,8580872,382.974335,579.569202


In [8]:
transfers.apply(lambda row: G.add_edge(row['to_stop_id'], row['from_stop_id'],
                                       trip_id=f"TRANSFER_{row['from_stop_id']}_{row['to_stop_id']}",
                                       t_time=row['t_time']), axis=1);

In [9]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 1880
Number of edges: 270417
Average in degree: 143.8388
Average out degree: 143.8388


#### 1.3 Save graph

In [10]:
# Save the graph
nx.write_gpickle(G, '../data/sbb_graph.gpickle')

In [11]:
# Save the stops
stops.to_parquet('../data/stops.parquet.gz', compression='gzip')

In [12]:
from orientexpress.delay import to_parquet
to_parquet()

bus_delays loaded:  3691 lines
bus_delays_without_time loaded:  1238 lines
tram_delays loaded:  552 lines
train_delays loaded:  472 lines
train_delays_without_type loaded:  282 lines


## 2. Testing the algorithm

In [3]:
!git lfs pull

In [4]:
# load for testing purposes
G = nx.read_gpickle('../data/sbb_graph.gpickle')
stops = pd.read_parquet('../data/stops.parquet.gz')
delay = Delay()

# Check orientexpress.planner, orientexpress.utils, orientexpress.delay

We test how well the algorithm performs, by comparing our proposed route to official CFF proposed routes and route outputs proposed on [Slack](https://app.slack.com/client/TST7DE1RN/C012AMGG10T/thread/C012AMGG10T-1589637712.147300) by the teaching team.

### 2.1 Test query: From Zürich HB (8503000) to Zürich, Auzelg (8591049), arrival by 12:30:00

#### Proposed routes from CFF:

<div style="text-align:center"><img src="../images/cff_screenshot_1.png" width="600" /></div>

Details of the latest route proposed by CFF:

<div style="text-align:center"><img src="../images/cff_screenshot_2.png" width="300"/></div>

Details of the second latest route proposed by CFF:

<div style="text-align:center"><img src="../images/cff_screenshot_3.png" width="300"/></div>


#### Proposed routes from teaching team:

Route 1 (corresponds to latest route proposed by CFF):
- 20.TA.26-9-A-j19-1.2.H: 8503000:0:41/42 at 12:07:00 ~ 8503310:0:3 at 12:17:00
- Walking: 8503310:0:3 ~ 8590620
- 168.TA.26-12-A-j19-1.2.H: 8590620 at 12:23:00 ~ 8591049 at 12:29:00

Route 2 (corresponds to second latest route proposed by CFF):
- 32.TA.80-159-Y-j19-1.8.H: 8503000:0:5 at 12:05:00 ~ 8503006:0:6 at 12:11:00
- Walking: 8503006:0:6 ~ 8580449
- 1914.TA.26-11-A-j19-1.27.R: 8580449 at 12:15:00 ~ 8591049 at 12:24:00

#### Proposed routes by Orient Express:
**50% threshold**
<div style="text-align:center"><img src="../images/hb_auzelg_50.png" width="800" /></div>

**95% threshold**
<div style="text-align:center"><img src="../images/hb_auzelg_95.png" width="800" /></div>

We propose routes with confidence thresholds set to 50% and 95%. We observe that the 50% one gives us the route proposed by the CFF and the teaching team. But as the train arrive at 12:29 and has an expected delay of 1'18", it makes sense that the probability of missing the last connection is high. Our second solution makes you leave slightly earlier but has a much higher probability of success. Awesome!

### 2.2 Test query: Zürich, ETH/Universitätsspital (8591123) Zürich, Enge (8503010), arrival by 18:00:00

#### Proposed routes from CFF:

<div style="text-align:center"><img src="../images/cff_screenshot_4.png" width="800" /></div>

#### Proposed route by Orient Express:

**50% threshold**
<div style="text-align:center"><img src="../images/ETH_Enge_50.png" width="800" /></div>

**90% threshold**
<div style="text-align:center"><img src="../images/ETH_Enge_90.png" width="800" /></div>

We propose a routes with the confidence threshold set to 50%. The route is the same as the last one proposed by the CFF. It seems like our results are pretty good. 

Increasing the threshold to 90% gives you a route that makes you leave 2 minutes earlier but gives you a 97% probability and you don't need to change! And it's not even suggested by the CFF.

### 2.3 Test query: Zürich, HB (8503000) Kloten, Kaserne Ost (8573233), arrival by 18:00:00

You're a [Richtstrahlpioner](https://www.miljobs.ch/fr/jobs-a-z/detail/job/111/show/) that needs to go back in service and don't want to get shouted at because you arrived late? We got you too!

#### Proposed routes from CFF:

<div style="text-align:center"><img src="../images/cff_hb_kloten.png" width="800" /></div>

#### Proposed route by Orient Express:

<div style="text-align:center"><img src="../images/hb_kloten_90.png" width="800" /></div>

We propose a routes with the confidence threshold set to 90%. The route is the same as the last one proposed by the CFF. This result seems pretty robust too.

### 2.4 Conclusion

In conclusion, we see that our results seem reasonable. We produce feasible plan, i.e. the ordering of the trips is correct. We don't suggest that you take trips that would make you travel back in time or that you can't realistically take. We take into account transfer time when computing the probabilities of catching the connections making it as realistic as possible... unlike the CFF that like you being stranded for 30 minutes in Bern when you can't catch the IR15 to Lausanne because the IC8 from Zürich is (always) delayed. Finally, increasing the probability threshold gives you slighty slower routes but where you can see which part of the proposed journey got improved on. We invite you to try out yourself by changing `/lab` to `/voila` in the URL and navigate to `notebooks/Viz.ipynb`. Bon voyage!